In [1]:
%reload_kedro
catalog = context.catalog

2021-08-24 12:06:21,054 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.3
2021-08-24 12:06:21,142 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-24 12:06:21,425 - root - INFO - ** Kedro project Spotify Recommendations
2021-08-24 12:06:21,426 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-08-24 12:06:21,435 - root - INFO - Registered line magic `run_viz`


In [3]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [4]:
client = d_dist.client._get_global_client() or Client()

In [2]:
cleaned_mpd_playlists = catalog.load("cleaned_mpd_playlists")

2021-08-24 12:06:21,461 - kedro.io.data_catalog - INFO - Loading data from `cleaned_mpd_playlists` (ParquetDataSet)...


In [5]:
pids_to_tids = cleaned_mpd_playlists[["pid", "track_ids"]].set_index("pid")["track_ids"]

In [6]:
from sklearn.model_selection import train_test_split
#We do not care for order mattering here as of now
#There is no user information so our task will focus on recommending songs to playlists
#  with expansion into optimizing ordering or playlists and then further playlist continuation where sequence matters

In [7]:
split_mpd_pids_to_tids = pids_to_tids.map(train_test_split).reset_index()

In [8]:
split_mpd_pids_to_tids.head(1)

,pid,track_ids
0,0,"[[4Y45aqo9QMa57rDsAJv40A, 0XUfyU2QviPAs6bxSpXY..."


In [9]:
split_mpd_pids_to_tids = split_mpd_pids_to_tids.assign(train=split_mpd_pids_to_tids["track_ids"].map(lambda x: x[0]),
                 test=split_mpd_pids_to_tids["track_ids"].map(lambda x: x[1]))

In [10]:
split_mpd_pids_to_tids.head(1)

,pid,track_ids,train,test
0,0,"[[5lDriBxJd22IhOH9zTcFrV, 6MjljecHzHelUDismyKk...","[5lDriBxJd22IhOH9zTcFrV, 6MjljecHzHelUDismyKkb...","[4E5P1XyAFtrjpiIxkydly4, 5Q0Nhxo0l2bP3pNjpGJwV..."


In [11]:
# split_mpd_pids_to_tids[["train_track_ids", "test_track_ids"]] = (pd.DataFrame(split_mpd_pids_to_tids["track_ids"].tolist(), 
#                                                                               index=split_mpd_pids_to_tids.index)
#                                                                 )

In [12]:
# train_mpd_pids_to_tids = split_pids_to_tids[["pid", "train_track_ids"]]
# test_mpd_pids_to_tids = split_pids_to_tids[["pid", "test_track_ids"]]

In [13]:
catalog.save("split_mpd_pids_to_tids", split_mpd_pids_to_tids)
# catalog.save("train_mpd_pids_to_tids", train_mpd_pids_to_tids)
# catalog.save("test_mpd_pids_to_tids", test_mpd_pids_to_tids)

2021-08-24 12:08:40,361 - kedro.io.data_catalog - INFO - Saving data to `split_mpd_pids_to_tids` (ParquetDataSet)...


Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/Users/i854336/.pyenv/versions/3.8.11/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/i854336/.pyenv/versions/3.8.11/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/.venv/lib/python3.8/site-packages/distributed/process.py", line 191, in _run
    target(*args, **kwargs)
  File "/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/.venv/lib/python3.8/site-packages/distributed/nanny.py", line 923, in _run
    loop.run_sync(do_stop)
  File "/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/.venv/lib/python3.8/site-packages/tornado/ioloop.py", line 524, in run_sync
    self.start()
  File "/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommen